In [11]:
import ultralytics
from ultralytics import YOLO
import os
from PIL import Image
import cv2
from IPython.display import Video
import glob
import matplotlib.pyplot as plt
import warnings
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

In [12]:
warnings.filterwarnings('ignore')

# Run ultralytics checks
ultralytics.checks()

Ultralytics YOLOv8.2.27 🚀 Python-3.8.10 torch-2.2.2 CPU (Apple M2)
Setup complete ✅ (8 CPUs, 8.0 GB RAM, 143.8/460.4 GB disk)


In [13]:
# Set the root directory for the dataset
Root_dir = 'dataset'

In [15]:
# Initialize the YOLO model with pre-trained weights
model = YOLO('yolov8s.pt')

In [16]:
# Set paths for the dataset YAML file and train/valid image directories
yaml_path = os.path.join(Root_dir, 'data.yaml')
train_path = os.path.join(Root_dir, 'train', 'images')
valid_path = os.path.join(Root_dir, 'valid', 'images')

In [17]:
# Define a custom Dataset class
class ChessDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.image_files = glob.glob(os.path.join(root_dir, '*.jpg'))
        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = self.image_files[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image

In [18]:
# Optimize data loading using DataLoader
def load_data(root_dir, batch_size=16, image_size=640):
    transform = transforms.Compose([
        transforms.Resize((image_size, image_size)),
        transforms.ToTensor(),
    ])
    dataset = ChessDataset(root_dir, transform=transform)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
    return dataloader

In [19]:
# Train the model with optimized configurations
def train_model(model, yaml_path, batch_size=16, epochs=50):
    model.train(data=yaml_path, imgsz=640, batch=batch_size, epochs=epochs, device='mps', workers=4)

In [20]:
# Optimize hardware utilization
# Ensure that MPS (Metal Performance Shaders) is being used
if torch.backends.mps.is_available():
    device = torch.device('mps')
    print("Using MPS:", torch.backends.mps.is_built())
else:
    device = torch.device('cpu')
    print("Using CPU")

Using MPS: True


In [21]:
# Load data
train_loader = load_data(train_path, batch_size=16, image_size=640)
valid_loader = load_data(valid_path, batch_size=16, image_size=640)

# Call the training function
train_model(model, yaml_path, batch_size=16, epochs=50)

Ultralytics YOLOv8.2.27 🚀 Python-3.8.10 torch-2.2.2 MPS (Apple M2)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=dataset/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=mps, workers=4, project=None, name=train8, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, 

[W NNPACK.cpp:64] Could not initialize NNPACK! Reason: Unsupported hardware.


Model summary: 225 layers, 11140631 parameters, 11140615 gradients, 28.7 GFLOPs

Transferred 349/355 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'


train: Scanning /Users/aaviix/Desktop/UNIVERSITY/SEMESTER 4/COMPUTER VISION/Chess-Pieces-Computer-Vision/dataset/train/labels.cache... 606 images, 0 backgrounds, 0 corrupt: 100%|██████████| 606/606 [00:00<?, ?it/s]

train: WARNING ⚠️ /Users/aaviix/Desktop/UNIVERSITY/SEMESTER 4/COMPUTER VISION/Chess-Pieces-Computer-Vision/dataset/train/images/26d663ab5ffbec49f9dc8e592982cfd4_jpg.rf.0fe973299ff9e73874d9e684cba4e406.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /Users/aaviix/Desktop/UNIVERSITY/SEMESTER 4/COMPUTER VISION/Chess-Pieces-Computer-Vision/dataset/train/images/26d663ab5ffbec49f9dc8e592982cfd4_jpg.rf.5e1a9d49ad09f5a7549225a05e1f66a6.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /Users/aaviix/Desktop/UNIVERSITY/SEMESTER 4/COMPUTER VISION/Chess-Pieces-Computer-Vision/dataset/train/images/26d663ab5ffbec49f9dc8e592982cfd4_jpg.rf.bb226be189ee3a6399fbbd5080852e5b.jpg: 1 duplicate labels removed



val: Scanning /Users/aaviix/Desktop/UNIVERSITY/SEMESTER 4/COMPUTER VISION/Chess-Pieces-Computer-Vision/dataset/valid/labels.cache... 58 images, 0 backgrounds, 0 corrupt: 100%|██████████| 58/58 [00:00<?, ?it/s]


Plotting labels to /Users/aaviix/Desktop/UNIVERSITY/SEMESTER 4/COMPUTER VISION/Chess-Pieces-Computer-Vision/runs/detect/train8/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000588, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to /Users/aaviix/Desktop/UNIVERSITY/SEMESTER 4/COMPUTER VISION/Chess-Pieces-Computer-Vision/runs/detect/train8
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/50         0G        nan        nan        nan        254        640:  21%|██        | 8/38 [12:22<29:12, 58.42s/it]   